## Train BERT model



In [1]:
%load_ext autoreload 
%autoreload 2
import os
import pandas as pd
from datasets import Dataset, Value, ClassLabel, Features
from pysentimiento.preprocessing import preprocess_tweet
from pysentimiento.tass import load_datasets


train_dataset, dev_dataset, test_dataset = load_datasets(
)


4802 2443 7264


In [3]:
train_dataset["label"]

[None,
 1,
 1,
 None,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 1,
 None,
 1,
 None,
 None,
 1,
 None,
 1,
 1,
 None,
 None,
 1,
 None,
 1,
 None,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 1,
 1,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 1,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 1,
 None,
 1,
 None,
 1,
 None,
 None,
 1,
 1,
 1,
 None,
 None,
 1,
 None,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 1,
 None,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 None,
 1,
 1,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 1,
 None,
 None,
 None,
 1,
 1,
 None,
 1,
 1,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 1,
 None,
 1,
 1,
 None,
 None

In [ ]:
import torch

from pysentimiento.tass import label2id, id2label, load_model

device = "cuda" if torch.cuda.is_available() else "cpu"
base_model = "dccuchile/bert-base-spanish-wwm-cased"

model, tokenizer = load_model(base_model, label2id=label2id, id2label=id2label)

model = model.to(device)
model.train();

In [6]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

batch_size = 32
eval_batch_size = 16

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
dev_dataset = dev_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)


In [7]:
def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['label']})
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)

In [20]:
train_dataset["labels"]

tensor([nan, 1., 1.,  ..., nan, nan, nan])

In [8]:
from transformers import TrainingArguments, Trainer

epochs = 3
warmup_proportion = 0.1


total_steps = (epochs * len(train_dataset)) // batch_size
warmup_steps = int(warmup_proportion * total_steps)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    warmup_steps=warmup_steps,
    evaluation_strategy="epoch",
    do_eval=False,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=lambda x: compute_metrics(x, id2label=id2label),
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

trainer.train()

ValueError: Target size (torch.Size([32])) must be the same as input size (torch.Size([32, 3]))

In [5]:
from pysentimiento import compute_metrics

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.781631,0.653295,0.641696,0.642772,0.650757


TrainOutput(global_step=151, training_loss=0.8435698250271627)

In [7]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.7347931861877441,
 'eval_accuracy': 0.686398678414097,
 'eval_f1': 0.6673146659667489,
 'eval_precision': 0.6676487102833413,
 'eval_recall': 0.6695717138336971,
 'epoch': 1.0}

In [ ]:
path = "../models/beto-sentiment-analysis"
model.save_pretrained(path)
tokenizer.save_pretrained(path)